This simple notebook is an update of the notebook by Jake Vanderplas at https://jakevdp.github.io/blog/2013/06/15/numba-vs-cython-take-2/ to run on Python 3.5. It compares usage of cython and numba illustrating the ability to vastly increase speed of computation with some simple tricks. 


In [1]:
%load_ext Cython

import numpy as np
X = np.random.random((1000, 3))
D = np.empty((1000, 1000))

In [2]:
# Pure python version

def pairwise_python(X, D):
    M = X.shape[0]
    N = X.shape[1]
    for i in range(M):
        for j in range(M):
            d = 0.0
            for k in range(N):
                tmp = X[i, k] - X[j, k]
                d += tmp * tmp
            D[i, j] = np.sqrt(d)

In [3]:
# numba version

import numpy as np
from numba import double
from numba.decorators import jit

@jit     #(arg_types=[double[:,:], double[:,:]])
def pairwise_numba(X, D):
    M = X.shape[0]
    N = X.shape[1]
    for i in range(M):
        for j in range(M):
            d = 0.0
            for k in range(N):
                tmp = X[i, k] - X[j, k]
                d += tmp * tmp
            D[i, j] = np.sqrt(d)

In [4]:
%%cython

cimport cython
from libc.math cimport sqrt

@cython.boundscheck(False)
@cython.wraparound(False)
def pairwise_cython(double[:, ::1] X, double[:, ::1] D):
    cdef int M = X.shape[0]
    cdef int N = X.shape[1]
    cdef double tmp, d
    for i in range(M):
        for j in range(M):
            d = 0.0
            for k in range(N):
                tmp = X[i, k] - X[j, k]
                d += tmp * tmp
            D[i, j] = sqrt(d)

In [5]:
%timeit pairwise_python(X, D)

3.04 s ± 70.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%timeit pairwise_numba(X, D)

2.91 ms ± 142 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%timeit pairwise_cython(X, D)

2.92 ms ± 105 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
